In [1]:
import pickle
import gzip
import scipy.stats as sps
import numpy as np
import os.path
import time

import dgl
from dgl.data.utils import download, extract_archive, get_download_dir

import rdkit
from rdkit.Chem import Descriptors
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem import rdmolops
from bo import sascorer
import networkx as nx

import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
from torch.distributions import constraints, transform_to

from jtnn import *


# from optparse import OptionParser
import argparse

torch.backends.cudnn.enabled=True

lg = rdkit.RDLogger.logger()
lg.setLevel(rdkit.RDLogger.CRITICAL)


# We define the functions used to load and save objects
def save_object(obj, filename):
    result = pickle.dumps(obj)
    with gzip.GzipFile(filename, 'wb') as dest: dest.write(result)
    dest.close()


def load_object(filename):
    with gzip.GzipFile(filename, 'rb') as source: result = source.read()
    ret = pickle.loads(result)
    source.close()
    return ret


# parser = OptionParser()
"""
parser = argparse.ArgumentParser()
parser.add_argument("-v", "--vocab", dest="vocab_path", default="/home/ubuntu/ASAIL/jtnn_bo/jtnn/vocab.txt")
parser.add_argument("-m", "--model", dest="model_path", required=True)
parser.add_argument("-o", "--save_dir", dest="save_dir", required=True)
parser.add_argument("-n", "--n_train", dest="training_num", default=500)
parser.add_argument("-i", "--n_ind", dest="inducing_num", default=500)
parser.add_argument("-w", "--hidden", dest="hidden_size", default=200)
parser.add_argument("-l", "--latent", dest="latent_size", default=56)
parser.add_argument("-d", "--depth", dest="depth", default=3)
parser.add_argument("-r", "--seed", dest="random_seed", default=19)
parser.add_argument("-e", "--evaluate", dest="eval", default=False)
"""

vocab_path = "/home/ubuntu/ASAIL/jtnn_bo/jtnn/vocab.txt"
model_path = "model.iter-0-1500"
save_dir = "result/"
hidden_size = 200
latent_size = 56
depth = 3
random_seed = 1
training_num = 20000
inducing_num = 500
eval = "False"


# We load the random seed
np.random.seed(int(random_seed))

# We load the data (y is minued!)
kkk = int(training_num)
M = int(inducing_num)
X = np.loadtxt('./bo/latent_features2.txt')[:kkk]
y = -np.loadtxt('./bo/targets2.txt')[:kkk]
y = y.reshape((-1, 1))
logP_values = np.loadtxt('./bo/logP_values2.txt')
SA_scores = np.loadtxt('./bo/SA_scores2.txt')
cycle_scores = np.loadtxt('./bo/cycle_scores2.txt')
SA_scores_normalized = (np.array(SA_scores) - np.mean(SA_scores)) / np.std(SA_scores)
logP_values_normalized = (np.array(logP_values) - np.mean(logP_values)) / np.std(logP_values)
cycle_scores_normalized = (np.array(cycle_scores) - np.mean(cycle_scores)) / np.std(cycle_scores)
#y = -logP_values
#y = y[:kkk].reshape((-1, 1))
#y = (np.array(y) - np.mean(y)) / np.std(y)


device = "cuda"
#device = "cpu"

n = X.shape[0]

permutation = np.random.choice(n, n, replace=False)

X_train = X[permutation, :][0: np.int(np.round(0.9 * n)), :]
X_test = X[permutation, :][np.int(np.round(0.9 * n)):, :]

y_train = y[permutation][0: np.int(np.round(0.9 * n))]
y_test = y[permutation][np.int(np.round(0.9 * n)):]

y_train = y_train.transpose()
y_test = y_test.transpose()


X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()


# Train sparse Gaussian by gpytorch:

import gpytorch
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal

class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.base_covar_module = ScaleKernel(RBFKernel())
        self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=train_x[:M, :], likelihood=likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)
    
    
y_train, y_test = y_train.reshape(-1), y_test.reshape(-1)
print(y_train.shape, y_test.shape)
    
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
model = GPRegressionModel(X_train, y_train, likelihood).to(device)

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

#training_iterations = 200
def train(training_iterations=20, optimizer=optimizer, X_train=X_train, y_train=y_train):
    for i in range(training_iterations):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(X_train)
        # Calc loss and backprop derivatives
        loss = -mll(output, y_train)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()
        #torch.cuda.empty_cache()
if eval == "False":
    with gpytorch.settings.use_toeplitz(True):
        train()

torch.Size([18000]) torch.Size([2000])
Iter 1/20 - Loss: 2.646
Iter 2/20 - Loss: 2.609
Iter 3/20 - Loss: 2.574
Iter 4/20 - Loss: 2.542
Iter 5/20 - Loss: 2.512
Iter 6/20 - Loss: 2.483
Iter 7/20 - Loss: 2.457
Iter 8/20 - Loss: 2.433
Iter 9/20 - Loss: 2.410
Iter 10/20 - Loss: 2.390
Iter 11/20 - Loss: 2.371
Iter 12/20 - Loss: 2.353
Iter 13/20 - Loss: 2.336
Iter 14/20 - Loss: 2.321
Iter 15/20 - Loss: 2.308
Iter 16/20 - Loss: 2.295
Iter 17/20 - Loss: 2.283
Iter 18/20 - Loss: 2.273
Iter 19/20 - Loss: 2.263
Iter 20/20 - Loss: 2.254


In [2]:
# Define helper function for BO
from scipy.stats import norm
import torch.nn.functional as F

def update_posterior(X_new, Y_new, iter=10):

    model.set_train_data(X_new, Y_new)
    # optimize the GP hyperparameters using Adam with lr=0.005
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    train(training_iteration=iter, optimizer=optimizer)

# Define Acquisition function
# TODO: the original paper uses expected imporvement 
def lower_confidence_bound(x, kappa=2):
    model.eval()
    mu, variance = model(x).mean, model(x).variance
    print(mu, variance)
    sigma = variance.sqrt()
    return mu - kappa * sigma

def find_a_candidate(x_init, lb, ub):
    # transform x to an unconstrained domain
    constraint = constraints.interval(lb, ub)
    #print(x_init)
    unconstrained_x_init = transform_to(constraint).inv(x_init)
    #print(unconstrained_x_init)
    unconstrained_x = unconstrained_x_init.clone().detach().requires_grad_(True)
    
    # WARNING: this is a memory intensive optimizer
    # TODO: Maybe try other gradient-based iterative methods
    minimizer = optim.LBFGS([unconstrained_x], max_iter=10)

    def closure():
        minimizer.zero_grad()
        x = transform_to(constraint)(unconstrained_x)
        y = lower_confidence_bound(x)
        #y = lower_confidence_bound(unconstrained_x)
        #print(autograd.grad(y, unconstrained_x))
        autograd.backward(unconstrained_x, autograd.grad(y, unconstrained_x))
        return y

    minimizer.step(closure)
    # after finding a candidate in the unconstrained domain,
    # convert it back to original domain.
    x = transform_to(constraint)(unconstrained_x)
    return x.detach()

def next_x(lb, ub, num_candidates_each_x=5, num_x=60):
    found_x=[]
    x_init = model.train_inputs[0][-1:]
    for j in range(num_x):
        candidates = []
        values = []
        for i in range(num_candidates_each_x):
            x = find_a_candidate(x_init, lb, ub)
            y = lower_confidence_bound(x)
            #print("next x:", x, x.shape)
            #print("next y:", y, y.shape)
            candidates.append(x)
            values.append(y)
            x_init = x.new_empty(1,56).uniform_(0, 1).mul(ub-lb).add_(lb).to(device)
            #print("next",x_init)
        argmin = torch.min(torch.cat(values), dim=0)[1].item()
        found_x.append(candidates[argmin])
        x_init=found_x[-1]
    return found_x

In [3]:
torch.backends.cudnn.benchmark = True
lb = torch.min(X_train, dim = 0)[0]
ub = torch.max(X_train, dim = 0)[0]
xmin = next_x(lb,ub,5,60)

tensor([0.0068], device='cuda:0', grad_fn=<ViewBackward>) tensor([1.2452], device='cuda:0', grad_fn=<ExpandBackward>)
tensor([0.0068], device='cuda:0', grad_fn=<ViewBackward>) tensor([1.2452], device='cuda:0', grad_fn=<ExpandBackward>)
tensor([0.0068], device='cuda:0', grad_fn=<ViewBackward>) tensor([1.2452], device='cuda:0', grad_fn=<ExpandBackward>)
tensor([0.0068], device='cuda:0', grad_fn=<ViewBackward>) tensor([1.2452], device='cuda:0', grad_fn=<ExpandBackward>)
tensor([0.0068], device='cuda:0', grad_fn=<ViewBackward>) tensor([1.2452], device='cuda:0', grad_fn=<ExpandBackward>)


RuntimeError: CUDA error: an illegal memory access was encountered

In [ ]:
# try gpytorch Bayesian optimization:
# Define helper function for BO
from scipy.stats import norm
import torch.nn.functional as F

def update_posterior(X_new, Y_new, iter=10):

    model.set_train_data(X_new, Y_new)
    # optimize the GP hyperparameters using Adam with lr=0.005
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    train(training_iteration=iter, optimizer=optimizer)

# Define Acquisition function
# TODO: the original paper uses expected imporvement 
def lower_confidence_bound(x, kappa=2):
    model.eval()
    pred = model(x)
    mu, variance = pred.mean, pred.variance
    print(mu, variance)
    sigma = variance.sqrt()
    print(mu, kappa, sigma)
    return mu - kappa * sigma

def expected_improvement(x):
    mu, variance = model(x, full_cov=False, noiseless=False)
    x_sample = model.X
    sigma = variance.sqrt()
    mu_sample, v_sample = model(X_sample, full_cov=False, noiseless=False)
    mu_sample_opt = torch.max(mu_sample)
    imp = mu - mu_sample
    z = imp / sigma
    ei = F.relu(imp) + sigma * norm.pdf(z) - torch.abs(imp) * norm.cdf(z)
    return ei
        
def find_a_candidate(x_init, lb, ub):

    def acquisition_minimizer(x_init, iterations=100):
        x = x_init.clone().requires_grad_(True)
        minimizer = optim.Adam([x], lr=0.01)
        for i in range(iterations):
            y = lower_confidence_bound(x)
            print("iteration - {}: lower_confidence_bound: {}".format(i, y))
            autograd.backward(x, autograd.grad(y, x), retain_graph=True)
            minimizer.step()
        return x
    x = acquisition_minimizer(x_init)
 
    print("minimizer finished")
    # after finding a candidate in the unconstrained domain,
    # convert it back to original domain.
    return x.detach()

def next_x(lb, ub, num_candidates_each_x=5, num_x=60):
    found_x=[]
    x_init = model.train_inputs[0][-1:]
    model.eval()
    print(model(x_init).mean, model(x_init).variance)
    for j in range(num_x):
        candidates = []
        values = []
        for i in range(num_candidates_each_x):
            print("start finding candidate")
            x = find_a_candidate(x_init, lb, ub)
            print(x)
            y = lower_confidencb
            #print("next x:", x, x.shape)
            #print("next y:", y, y.shape)
            candidates.append(x)
            values.append(y)
            x_init = x.new_empty(1,56).uniform_(0, 1).mul(ub-lb).add_(lb).to(device)
            #print("next",x_init)
        argmin = torch.min(torch.cat(values), dim=0)[1].item()
        found_x.append(candidates[argmin])
        x_init=found_x[-1]
    return found_x

In [ ]:
lb = torch.min(X_train, dim = 0)[0]
ub = torch.max(X_train, dim = 0)[0]

In [ ]:
xmin = next_x(lb,ub,5,60)

In [ ]:
model.eval()
print(model(X_train[:1]).loc)
print(model(X_train[:1]).mean)